<center><h1>Replication Prototype – Monopoly Setting</h1></center>

In [ ]:
# Standard library
import asyncio
import json
import os
import sys

sys.path.append(os.path.abspath(os.path.join("../..")))

# Third-party libraries
import matplotlib.pyplot as plt
import nest_asyncio
from dotenv import load_dotenv

# Apply third-party setup
nest_asyncio.apply()

# Local application imports
from utils.prompts import GENERAL_PROMPT, PP_P0
from utils.pricing_market_logic_multiproduct import (
    get_monopoly_prices,
    get_nash_prices,
    get_profits,
    get_quantities,
)
from utils.utils import (
    create_output_paths,
    save_round_data,
    simulate_n_firms_round,
    update_plot,
)

load_dotenv()

# Configuration
API_KEY = os.getenv("MISTRAL_API_KEY")
MODEL_NAME = "magistral-small-2506"  # Free open-source model


# Experiments
---

Notes: 

- For each $\alpha \in \{1,3.2,10\}$ 
- For each LLM, we conduct a **300-period** run using the **P0 prompt prefix** in a monopoly setting.
- **Valid output**: syntactically conforms to the instructions in the prompt. 
- **Convergence to a price p**: means that in periods 201–300, the top 90th percentile and bottom 10th percentile prices are within 5% of p. 
- **Monopoly price**: pM is the price p1 that maximizes the profit π1 = (p1−αc1)·q1.

In [ ]:
N_FIRMS = 1
MU, BETA, SIGMA = 0.25, 100, 0 #Follows Calvano et al. (2020b)
C_i, A_i, A_0, MG_C = 1, 2, 0, 1.0
PROMPT_PREFIX = PP_P0
N_PERIODS = 300

for ALPHA in [1, 3.2, 10]:  
    WILLIWGNES_TO_PAY = 4.51 * ALPHA
    A = tuple([A_i for _ in range(N_FIRMS)])
    ALPHA = tuple([ALPHA for _ in range(N_FIRMS)])
    C = tuple([C_i for _ in range(N_FIRMS)])
    group_idxs = tuple([i for i in range(1, N_FIRMS+1)])

    p_m = get_monopoly_prices(
        a0=A_0,
        a=A,
        mu=MU,
        alpha=ALPHA,
        c=C,
        multiplier=BETA,
        sigma=SIGMA,
        group_idxs=group_idxs,
    )

    q_m = get_quantities(
        p=tuple(p_m),
        a0=A_0,
        a=A,
        mu=MU,
        alpha=ALPHA,
        multiplier=BETA,
        sigma=SIGMA,
        group_idxs=group_idxs,
    )

    pi_m = get_profits(
        p=tuple(p_m),
        a0=A_0,
        a=A,
        mu=MU,
        alpha=ALPHA,
        c=C,
        multiplier=BETA,
        sigma=SIGMA,
        group_idxs=group_idxs,
    )


    print(f"Monopoly prices: {p_m} | Quantities: {q_m} | Profits: {pi_m}")


    firm_names = [f"firm_{i+1}" for i in range(N_FIRMS)]
    sub_path = f"monopoly/alpha_{int(ALPHA[0])}"
    paths = create_output_paths(sub_path=sub_path, 
                                model_name=MODEL_NAME, 
                                firm_names=firm_names
                                )

    plans = {firm: "No previous plans." for firm in firm_names}
    insights = {firm: "No previous insights." for firm in firm_names}
    market_data = {firm: "No previous market data." for firm in firm_names}

    # Initialize the market data file
    experiment_results={'price_history': {k:[] for k in firm_names},
                        'quantity_history': {k:[] for k in firm_names},
                        'profit_history': {k:[] for k in firm_names},
                        'time_history': [],
                        }

    fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    plt.tight_layout()

    for i in range(1, N_PERIODS+1):
        
        # 1) Generate dict with prompts per firm
        agent_prompt_dict = {firm_name: {'prompt':GENERAL_PROMPT.format(#prompt_prefix=PROMPT_PREFIX,
                                                        marginal_cost=MG_C,
                                                        willigness_to_pay=WILLIWGNES_TO_PAY,
                                                        previous_plans = plans[firm_name],
                                                        previous_insights = insights[firm_name],
                                                        market_data = market_data[firm_name]
                                                        ),
                                        'prefix': PROMPT_PREFIX,
                                        }
                            
                            for firm_name in firm_names}
        
        #Firms act simultaneously (async prompts)
        results = asyncio.run(simulate_n_firms_round(agent_prompt_dict))
        firm_responses = {result['firm']: result['response'] for result in results}
        
        #Get chosen prices
        prices = [firm_responses[firm]['chosen_price'] for firm in firm_names]

        # Compute quantities and profits for each firm, based on the chosen prices
        quantities = get_quantities(
                        p=tuple(prices),
                        a0=A_0,
                        a=A,
                        mu=MU,
                        alpha=ALPHA,
                        multiplier=BETA,
                        sigma=SIGMA,
                        group_idxs=group_idxs,
                    )

        profits = get_profits(
                    p=tuple(prices),
                    a0=A_0,
                    a=A,
                    mu=MU,
                    alpha=ALPHA,
                    c=C,
                    multiplier=BETA,
                    sigma=SIGMA,
                    group_idxs=group_idxs,
                )
        # Save the results for each firm
        for firm in firm_names:
            experiment_results['price_history'][firm].append(prices[firm_names.index(firm)])
            experiment_results['quantity_history'][firm].append(quantities[firm_names.index(firm)])
            experiment_results['profit_history'][firm].append(profits[firm_names.index(firm)])

        experiment_results['time_history'].append(i)

        # Save responses + logs
        firm_responses = save_round_data(i, paths, firm_responses, prices, quantities, profits, firm_names)

        # Update prompt components
        plans = {firm: {firm_responses[firm]['plans']} for firm in firm_names}
        insights = {firm: {firm_responses[firm]['insights']} for firm in firm_names}
        market_data = {firm: firm_responses[firm]['last_market_data'] for firm in firm_names}

        update_plot(
            fig, axs,
            p_m=p_m, q_m=q_m, pi_m=pi_m, alpha=ALPHA,
            price_history=experiment_results["price_history"],
            quantity_history=experiment_results["quantity_history"],
            profit_history=experiment_results["profit_history"],
            time_history=experiment_results["time_history"],
            model_name=MODEL_NAME,
            start_time=paths["start_time"],
            save_path=paths["plot"] + f"/{i:03d}_experiment_round.png",
            prompt_number=1,
            fill_convergende_range=True
        )


    # Turn off interactive plotting and show final plot
    plt.close(fig)

    # Save the final results to a JSON file
    with open(paths["output_dir"] + "/results.json", 'w') as f:
        json.dump(experiment_results, f, indent=4)